# stamps - Extended Feature Selection
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Feb 25, 2022

## Hypothesis
Two very important components of a machine learning model are feature selection and feature engineering. Our idea is that adding some more features to the StellarAlgo retention model could improve performance of the model. This notebook will test the standard StellarAlgo retention model features.

## Experiment
This section details our experiment including querying data, data transformations, feature selection and modelling.

In [1]:
import getpass
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from pycaret.classification import *

warnings.filterwarnings('ignore')

### Let's connect to MSSQL and run a stored proc to get our dataset:

In [2]:
# connect to SQL Server.
SERVER = '34.206.73.189'  
DATABASE = 'datascience' 
USERNAME = 'nrad' 
PASSWORD = '83F25619-D272-4660-98A2-93AF5CC18D59'#getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

In [3]:
lkupclientid = 35 # stamps

cursor = CNXN.cursor()

storedProc = (
    f"""Exec [stlrCFL].[ds].[getRetentionScoringModelData] {lkupclientid}"""
)

df = pd.read_sql(storedProc, CNXN)

# apply some data transformations
df["year"] = pd.to_numeric(df["year"])

CNXN.commit()
cursor.close()

df.head()
#df.info()

,lkupClientId,dimCustomerMasterId,customerNumber,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,source_tenure,tenure,distToVenue,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,click_link,fill_out_form,open_email,send_email,unsubscribe_email,openToSendRatio,clickToSendRatio,clickToOpenRatio,posting_records,resale_records,resale_atp,forward_records,cancel_records,email,inbound_email,inbound_phonecall,inperson_contact,internal_note,left_message,outbound_email,outbound_phonecall,phonecall,text,unknown,gender,childrenPresentInHH,maritalStatus,lengthOfResidenceInYrs,annualHHIncome,education,urbanicity,credits_after_refund,is_Lockdown,NumberofGamesPerSeason,CNTPostponedGames,isNextYear_Buyer
0,35,326205248,1136572,2017,Full Season,1068.93,2017-11-03,1.000000,241,TRUE,2190,368,6.34,8,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,9,None,1
1,35,326205306,661041,2017,Full Season,663.39,2017-09-29,0.625000,164,TRUE,3650,291,178.37,5,1,1,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,9,None,1
2,35,326205402,123264,2017,Full Season,2001.78,2017-10-20,0.833333,210,TRUE,4380,337,19.17,8,0,1,0,1,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,9,None,1
3,35,326205660,1028431,2017,Full Season,475.02,2017-11-03,0.875000,210,TRUE,2555,337,19.17,9,0,1,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,9,None,1
4,35,326205736,1274993,2017,Full Season,1182.96,2017-09-16,0.750000,35,TRUE,1825,162,9.43,6,2,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,9,None,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24759 entries, 0 to 24758
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lkupClientId            24759 non-null  int64  
 1   dimCustomerMasterId     24759 non-null  int64  
 2   customerNumber          24759 non-null  object 
 3   year                    24759 non-null  int64  
 4   productGrouping         24759 non-null  object 
 5   totalSpent              24759 non-null  float64
 6   recentDate              24759 non-null  object 
 7   attendancePercent       24759 non-null  float64
 8   renewedBeforeDays       24759 non-null  int64  
 9   isBuyer                 24759 non-null  object 
 10  source_tenure           24759 non-null  int64  
 11  tenure                  24759 non-null  int64  
 12  distToVenue             24759 non-null  float64
 13  totalGames              24759 non-null  int64  
 14  recency                 24759 non-null

### Let's also drop the features that only have a single value, as they won't add much differentiation to our model:

In [5]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)
        
df.shape

(24759, 31)

### We should also drop features that have a low correlation with the target label as they won't be useful for prediction, we'll only keep features that have a correlation above a set threshold:

In [6]:
cor = df.corr()

threshold = 0.05

#Correlation with output variable
cor_target = abs(cor["isNextYear_Buyer"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target > threshold]

feats = []
for name, val in relevant_features.items():
    feats.append(name)

df_correlated = df[feats]

df_correlated.shape

df_correlated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24759 entries, 0 to 24758
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   dimCustomerMasterId  24759 non-null  int64  
 1   year                 24759 non-null  int64  
 2   attendancePercent    24759 non-null  float64
 3   renewedBeforeDays    24759 non-null  int64  
 4   source_tenure        24759 non-null  int64  
 5   tenure               24759 non-null  int64  
 6   distToVenue          24759 non-null  float64
 7   totalGames           24759 non-null  int64  
 8   recency              24759 non-null  int64  
 9   missed_games_over_2  24759 non-null  int64  
 10  isNextYear_Buyer     24759 non-null  int64  
dtypes: float64(2), int64(9)
memory usage: 2.1 MB


### Now that we have the right features we can look at the correlations between them, if features are highly correlated with each other it might negatively impact the model:

In [7]:
corr = df_correlated.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,dimCustomerMasterId,year,attendancePercent,renewedBeforeDays,source_tenure,tenure,distToVenue,totalGames,recency,missed_games_over_2,isNextYear_Buyer
dimCustomerMasterId,1.00,0.20,0.06,-0.03,0.08,0.20,-0.02,0.05,-0.05,-0.02,0.35
year,0.20,1.00,-0.18,-0.66,-0.13,0.86,0.00,-0.44,0.08,0.06,-0.05
attendancePercent,0.06,-0.18,1.00,0.22,0.10,-0.12,-0.05,0.79,-0.43,-0.59,0.21
renewedBeforeDays,-0.03,-0.66,0.22,1.00,0.30,-0.44,-0.05,0.48,-0.10,-0.09,0.21
source_tenure,0.08,-0.13,0.10,0.30,1.00,0.20,-0.10,0.19,-0.04,-0.03,0.24
tenure,0.20,0.86,-0.12,-0.44,0.20,1.00,-0.03,-0.31,0.06,0.05,0.06
distToVenue,-0.02,0.00,-0.05,-0.05,-0.10,-0.03,1.00,-0.04,-0.00,0.08,-0.07
totalGames,0.05,-0.44,0.79,0.48,0.19,-0.31,-0.04,1.00,-0.42,-0.33,0.28
recency,-0.05,0.08,-0.43,-0.10,-0.04,0.06,-0.00,-0.42,1.00,0.25,-0.07
missed_games_over_2,-0.02,0.06,-0.59,-0.09,-0.03,0.05,0.08,-0.33,0.25,1.00,-0.08


### In order to compare two sets of features, we need to create some datasets for training and evalution:


In [8]:
# select % of the data for training
df_train = df_correlated.sample(frac=0.8, random_state=786).reset_index(drop=True)

# create the eval datasets for A and B
df_eval = df_correlated.drop(df_train.index).reset_index(drop=True)

# print out the number of records for training and eval
print('Data for Modeling: ' + str(df_train.shape))
print('Unseen Data For Predictions: ' + str(df_eval.shape), end="\n\n")

Data for Modeling: (19807, 11)
Unseen Data For Predictions: (4952, 11)



### Now we can model the data using a binary classification prediction for the isnextyear_buyer field to see how likely a customer is to re-purchase.

In [9]:
setup(
    data= df_train, 
    target="isNextYear_Buyer", 
    train_size = 0.80,
    data_split_shuffle=True,
    silent=True,
    numeric_features=[
        "attendancePercent",
        "renewedBeforeDays",
        "source_tenure",
        "tenure",
        "distToVenue",
        "totalGames",
        "recency",
        "missed_games_over_2"
    ]
)

,Description,Value
0,session_id,7099
1,Target,isNextYear_Buyer
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(19807, 11)"
5,Missing Values,False
6,Numeric Features,9
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


(       dimCustomerMasterId  attendancePercent  renewedBeforeDays  \
 15514          346137152.0           0.781250              197.0   
 4730           346212736.0           1.000000              164.0   
 13811          346211008.0           1.000000              151.0   
 773            346211712.0           1.125000               45.0   
 13180          346197504.0           0.833333               57.0   
 ...                    ...                ...                ...   
 9772           346209312.0           1.000000               57.0   
 1240           346211680.0           0.916667               62.0   
 3671           326320064.0           0.656250              158.0   
 10229          346201504.0           0.625000              225.0   
 19592          346133952.0           0.875000              160.0   
 
        source_tenure  tenure  distToVenue  totalGames  recency  \
 15514         3285.0   697.0     0.990000         7.0      0.0   
 4730          4015.0   291.0    27.

In [10]:
model_matrix = compare_models(
    fold=10,
    include=["lr", "xgboost"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.8911,0.9238,0.9486,0.9208,0.9345,0.6128,0.6154,0.7230
lr,Logistic Regression,0.8185,0.8196,1.0000,0.8185,0.9002,0.0000,0.0000,0.3790


In [11]:
best_model = create_model(model_matrix)
final_model = finalize_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8839,0.9203,0.9468,0.9144,0.9303,0.5838,0.5869
1,0.8940,0.9367,0.9468,0.9254,0.9360,0.6285,0.6299
2,0.8927,0.9151,0.9584,0.9146,0.9360,0.6064,0.6127
3,0.8877,0.9283,0.9468,0.9185,0.9324,0.6008,0.6032
4,0.8946,0.9208,0.9476,0.9255,0.9364,0.6302,0.6317
5,0.8895,0.9259,0.9453,0.9218,0.9334,0.6102,0.6119
6,0.8984,0.9289,0.9561,0.9226,0.9390,0.6341,0.6378
7,0.8857,0.9161,0.9453,0.9177,0.9313,0.5934,0.5957
8,0.8813,0.9132,0.9437,0.9141,0.9287,0.5758,0.5784
9,0.9034,0.9328,0.9498,0.9333,0.9415,0.6649,0.6658


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

## Observations
Here you can document some ideas on the results from above

## Conclusions
Here you can talk about next steps, did the experiment work? If yes, what to do next? If no, why?